In [6]:
import time
from datetime import date
import hmac
from requests import Request, Session
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#API
def api(prepared):
    ts = int(time.time() * 1000)
    signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
    signature = hmac.new('C0aO3tL7ukLZSf9BmDEdNOCyySgRxAocwExwKfy'.encode(), signature_payload, 'sha256').hexdigest()#シークレットキー
    prepared.headers['FTX-KEY'] = 'A6yJp66i2z3EFVnvC-8RFvmzVn9f6BKGKtBPYEwSV'#APIキー
    prepared.headers['FTX-SIGN'] = signature
    prepared.headers['FTX-TS'] = str(ts)
    return prepared

#全銘柄
def crypts():
    prepared = Request('GET', 'https://ftx.com/api/markets').prepare()
    Api = api(prepared)
    json = Session().send(Api).json()['result']
    
    df = pd.DataFrame(json)
    df = df.loc[:, ['name']]#826
    crs = df.query('name.str.endswith("/USD") | name.str.endswith("/JPY")', engine='python')#427 
    crs = crs.query('(~name.str.contains("BEAR") & ~name.str.contains("BULL") & ~name.str.contains("HALF") & ~name.str.contains("HEDGE"))', engine='python')#260
    return crs['name']

#データ(close, volume)
def cr_df(crypt):
    prepared = Request('GET', 'https://ftx.com/api/markets/'+crypt+'/candles?resolution=3600').prepare()
    Api = api(prepared)
    json = Session().send(Api).json()['result']
    
    df = pd.DataFrame(json)
    df = df[['startTime', 'close', 'volume']]
    df['startTime'] = pd.to_datetime(df['startTime'])
    df = df.rename(columns = {'startTime' : 'date'})
    df = df.set_index('date')
    return df

#取引量上位銘柄データ（数=cr_num）
def df(crypts, cr_num):
    li_vol = []
    li_price = []
    for crypt in crypts:
        try:
            Cr_df = cr_df(crypt)
            ave = np.average(Cr_df['volume'])
            vol = pd.DataFrame({'name' : [crypt], 'vol_ave' : [ave]})
            li_vol.append(vol)
        
            Cr_df = Cr_df.drop(['volume'], axis=1)
            price = Cr_df.rename(columns = {'close' : crypt})
            li_price.append(price)
        except(KeyError):
            continue
        
    df_vol = pd.concat(li_vol)
    Lcr = df_vol.nlargest(cr_num, "vol_ave", keep="all")['name']
    
    df_price = pd.concat(li_price, axis=1)
    df = df_price[Lcr]
    return df

#上位10銘柄データ
def Main_FTX(num):
        Crypts = crypts()
        Df = df(Crypts, num)
        return Df

        
Main_FTX(10)

,BTC/USD,ETH/USD,USDT/USD,SOL/USD,XRP/USD,BNB/USD,BTC/JPY,USD/JPY,AVAX/USD,MATIC/USD
date,,,,,,,,,,
2022-05-07 21:00:00+00:00,35030.0,2620.6,0.9998,78.3950,0.577750,363.967,4583095.22,NaN,54.8125,1.012144
2022-05-07 22:00:00+00:00,35039.0,2602.6,0.9998,77.8200,0.574750,362.330,4583190.75,NaN,53.7920,1.004622
2022-05-07 23:00:00+00:00,35462.0,2635.1,0.9998,78.8225,0.580700,365.243,4632199.31,NaN,55.0390,1.019203
2022-05-08 00:00:00+00:00,35092.0,2617.9,0.9998,78.4050,0.578250,363.436,4585918.49,NaN,54.3170,1.011398
2022-05-08 01:00:00+00:00,34810.0,2594.6,0.9998,77.3100,0.577225,361.439,4552374.38,NaN,53.2840,0.999928
2022-05-08 02:00:00+00:00,34568.0,2549.2,0.9996,76.9700,0.574000,358.178,4521682.99,NaN,53.2085,0.988944
2022-05-08 03:00:00+00:00,34469.0,2545.0,0.9996,76.8875,0.574325,359.120,4520933.51,NaN,53.4315,0.989847
2022-05-08 04:00:00+00:00,34398.0,2535.0,0.9997,77.2425,0.570375,358.085,4498627.29,NaN,53.4375,0.984937
2022-05-08 05:00:00+00:00,34446.0,2537.4,0.9996,77.6900,0.570200,359.046,4504915.25,NaN,53.8165,0.985413
